In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Data Scraping Code 

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
html = requests.get(url).text

In [4]:
soup = BeautifulSoup(html)

In [5]:
table = soup.select('table')[0]
tr_list = table.select('tr')
rows = []
for tr in tr_list:
    td_list = tr.select('td')
    tdl = []
    for td in td_list:
        tdl.append(td.text.strip())
    if tdl:
        rows.append(tdl)

Code for extracting the table data

In [6]:
rows

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 

In [7]:
df=pd.DataFrame(rows)

In [8]:
df.head()

,0,1,2
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
df.rename(columns={0:'Postal Code',1:'Borough',2:'Neighbourhood'},inplace=True)

In [10]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
df.drop(df[df['Borough']=='Not assigned'].index,axis=0,inplace=True)

droping not assinged cells

In [12]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


printing of dataframe

In [13]:
print(df)

    Postal Code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M5A  Downtown Toronto   
6           M6A        North York   
7           M6A        North York   
8           M7A      Queen's Park   
10          M9A         Etobicoke   
11          M1B       Scarborough   
12          M1B       Scarborough   
14          M3B        North York   
15          M4B         East York   
16          M4B         East York   
17          M5B  Downtown Toronto   
18          M5B  Downtown Toronto   
19          M6B        North York   
22          M9B         Etobicoke   
23          M9B         Etobicoke   
24          M9B         Etobicoke   
25          M9B         Etobicoke   
26          M9B         Etobicoke   
27          M1C       Scarborough   
28          M1C       Scarborough   
29          M1C       Scarborough   
31          M3C        North York   
32          M3C        North York   
3

In [14]:
df2=df.groupby(by=['Postal Code','Borough']).agg(lambda x:','.join(x))
df2.reset_index(level=['Postal Code','Borough'],inplace=True)

In [15]:
df2.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
df2[df2['Neighbourhood']=='Not assigned']=df2['Borough']

In [17]:
df2.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [18]:
df2.shape

(103, 3)

In [19]:
df3=pd.read_csv(r'C:\Users\Varun Kurup\Downloads\Geospatial_Coordinates.csv')

In [20]:
df4=pd.merge(df2,df3,on='Postal Code',how='left')

In [21]:
df4.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [22]:
import folium
from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


visualizing the neighbourhood

In [23]:
from IPython.display import display
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)
df4=df4.dropna(subset=['Longitude'])
df4=df4.dropna(subset=['Latitude'])

# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
display(map_toronto)

In [25]:
tordf=df4[df4['Borough']=='Central Toronto'].reset_index(drop=True)

grouping neighbourhoods near CentrLal Toronto

In [26]:
tordf.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160


In [27]:
import folium
from geopy.geocoders import Nominatim
address = 'Central Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location1 = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude1, longitude1))




The geograpical coordinate of Toronto are 43.653963, -79.387207.


creating the maps for the neighbourhood

In [28]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=10)
tordf=tordf.dropna(subset=['Longitude'])
tordf=tordf.dropna(subset=['Latitude'])
# add markers to map
for lat, lng, borough, neighborhood in zip(tordf['Latitude'], tordf['Longitude'], tordf['Borough'], tordf['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [29]:
tordf.loc[0,'Longitude']

-79.3887901

In [30]:
neighbourhood_latitude=tordf.loc[0,'Latitude']
neighbourhood_longitude=tordf.loc[0,'Longitude']

Foursquare credentials

In [31]:
CLIENT_ID='DRKAM5FSF35B5KQH4JOAAHQDM53J523UT5AP3JIPUX2AUT2O'
CLIENT_SECRET='UX3HFQXWU4AP5MCWRBIHPPDQIRIVJDK0XWHVTLD1FENHOM1L'
VERSION='20190614'

In [32]:
LIMIT=300
radius=500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,neighbourhood_latitude,neighbourhood_longitude,radius,LIMIT)

In [33]:
results=requests.get(url).json()
print(results)

{'meta': {'code': 200, 'requestId': '5d0542d39fb6b724b20c333c'}, 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."}, 'headerLocation': 'Toronto', 'headerFullLocation': 'Toronto', 'headerLocationGranularity': 'city', 'totalResults': 3, 'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273}, 'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '50e6da19e4b0d8a78a0e9794', 'name': 'Lawrence Park Ravine', 'location': {'address': '3055 Yonge Street', 'crossStreet': 'Lawrence Avenue East', 'lat': 43.72696303913755, 'lng': -79.39438246708775, 'labeledLatLngs': [{'label': 'display', 'lat': 43.72696303913755, 'lng': -79.39438246708775}], 'distance': 465

In [34]:
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list=row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Converting the json file to dataframe

In [35]:
import json
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
nearby_venues=json_normalize(venues)
filtered_coloumns=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_coloumns]
nearby_venues['venue.categories']=nearby_venues.apply(get_category_type,axis=1)
nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    
   

here the top neighbourhoods based on the data are grouped in a dataframe

In [37]:
toronto_venues = getNearbyVenues(names=tordf['Neighbourhood'],
                                   latitudes=tordf['Latitude'],
                                   longitudes=tordf['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville


In [38]:
toronto_venues.shape

(111, 7)

In [39]:
toronto_venues['Venue Category'].unique()[:100]

array(['Park', 'Swim School', 'Bus Line', 'Food & Drink Shop',
       'Breakfast Spot', 'Hotel', 'Grocery Store', 'Sandwich Place',
       'Gym', 'Clothing Store', 'Playground', 'Yoga Studio', 'Diner',
       'Coffee Shop', 'Mexican Restaurant', 'Salon / Barbershop',
       'Chinese Restaurant', 'Spa', 'Dessert Shop',
       'Fast Food Restaurant', 'Sporting Goods Shop', 'Bagel Shop',
       'Gym / Fitness Center', 'Rental Car Location', 'Café',
       'Indian Restaurant', 'Pizza Place', 'Seafood Restaurant',
       'Italian Restaurant', 'Sushi Restaurant', 'Brewery',
       'Thai Restaurant', 'Toy / Game Store', 'Restaurant',
       'Greek Restaurant', 'Gourmet Shop', 'Pharmacy', 'Farmers Market',
       'Flower Shop', 'Costume Shop', 'Discount Store',
       'Fried Chicken Joint', 'Deli / Bodega', 'Tennis Court',
       'Liquor Store', 'Supermarket', 'American Restaurant', 'Sports Bar',
       'Pub', 'Vietnamese Restaurant', 'Light Rail Station', 'Garden',
       'Ice Cream Shop', 'T

grouping the top venues together 

In [40]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,...,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,0.000000,0.052632,0.0000,...,0.000000,0.052632,0.000000,0.0,0.052632,0.026316,0.00,0.000000,0.000000,0.0000
1,Davisville North,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0000
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.071429,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.071429,0.071429,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.071429,0.0000
3,"Forest Hill North,Forest Hill West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.250000,0.000000,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.0000
4,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0000,...,0.000000,0.000000,0.333333,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0000
5,"Moore Park,Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.00,0.000000,0.000000,0.0000
6,North Toronto West,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0625
7,Roselawn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.0000
8,"The Annex,North Midtown,Yorkville",0.043478,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.130435,0.0000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.043478,0.000000,0.0000


displaying top 5 frequent venues

In [42]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
             venue  freq
0      Pizza Place  0.11
1   Sandwich Place  0.08
2     Dessert Shop  0.08
3  Thai Restaurant  0.05
4       Restaurant  0.05


----Davisville North----
               venue  freq
0     Clothing Store  0.11
1               Park  0.11
2  Food & Drink Shop  0.11
3         Playground  0.11
4      Grocery Store  0.11


----Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West----
                 venue  freq
0                  Pub  0.14
1          Coffee Shop  0.14
2  American Restaurant  0.07
3           Sports Bar  0.07
4           Bagel Shop  0.07


----Forest Hill North,Forest Hill West----
                 venue  freq
0        Jewelry Store  0.25
1                Trail  0.25
2                 Park  0.25
3     Sushi Restaurant  0.25
4  American Restaurant  0.00


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1                 Park  0.33
2          Swim School  0.33
3  American Restaurant  0.00
4   Salo

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Restaurant,Thai Restaurant,Sushi Restaurant,Café,Italian Restaurant,Coffee Shop,Discount Store
1,Davisville North,Hotel,Food & Drink Shop,Sandwich Place,Park,Clothing Store,Playground,Breakfast Spot,Grocery Store,Gym,Gourmet Shop
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Pub,Coffee Shop,American Restaurant,Sushi Restaurant,Liquor Store,Light Rail Station,Pizza Place,Supermarket,Sports Bar,Fried Chicken Joint
3,"Forest Hill North,Forest Hill West",Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Discount Store,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio
4,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Diner,Gym,Grocery Store,Greek Restaurant,Gourmet Shop,Garden


Clustering the data

In [45]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
print(kmeans.labels_)

[0 0 0 4 3 2 0 1 0]


In [46]:
coffee_shop=toronto_grouped[["Neighborhood","Coffee Shop"]]

Segmenting based on the coffee shop data 

In [47]:
coffee_shop.head()
print(coffee_shop)

                                        Neighborhood  Coffee Shop
0                                         Davisville     0.052632
1                                   Davisville North     0.000000
2  Deer Park,Forest Hill SE,Rathnelly,South Hill,...     0.142857
3                 Forest Hill North,Forest Hill West     0.000000
4                                      Lawrence Park     0.000000
5                         Moore Park,Summerhill East     0.000000
6                                 North Toronto West     0.125000
7                                           Roselawn     0.000000
8                  The Annex,North Midtown,Yorkville     0.130435


In [49]:
toclusters = 3

to_clustering = coffee_shop.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 1, 0, 0, 0, 1, 0, 1])

In [51]:
to_merged = coffee_shop.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [52]:
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(111, 9)


,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Davisville,0.052632,2,43.704324,-79.38879,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café
0,Davisville,0.052632,2,43.704324,-79.38879,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
0,Davisville,0.052632,2,43.704324,-79.38879,XO Gelato,43.705177,-79.388793,Dessert Shop
0,Davisville,0.052632,2,43.704324,-79.38879,Marigold Indian Bistro,43.702881,-79.388008,Indian Restaurant
0,Davisville,0.052632,2,43.704324,-79.38879,Viva Napoli,43.705752,-79.389125,Pizza Place


In [53]:
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(111, 9)


,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Davisville North,0.000000,0,43.712751,-79.390197,Subway,43.708378,-79.390473,Sandwich Place
7,Roselawn,0.000000,0,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
7,Roselawn,0.000000,0,43.711695,-79.416936,Menchie's St. Clair West,43.707664,-79.414301,Ice Cream Shop
1,Davisville North,0.000000,0,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
1,Davisville North,0.000000,0,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
1,Davisville North,0.000000,0,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
1,Davisville North,0.000000,0,43.712751,-79.390197,Best Western Roehampton Hotel & Suites,43.708878,-79.390880,Hotel
1,Davisville North,0.000000,0,43.712751,-79.390197,Circle K,43.712834,-79.391554,Grocery Store
1,Davisville North,0.000000,0,43.712751,-79.390197,Gym,43.713126,-79.393537,Gym
1,Davisville North,0.000000,0,43.712751,-79.390197,Winners,43.713236,-79.393873,Clothing Store


In [54]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [55]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [56]:
to_merged.loc[to_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Davisville North,0.0,0,43.712751,-79.390197,Subway,43.708378,-79.390473,Sandwich Place
7,Roselawn,0.0,0,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
7,Roselawn,0.0,0,43.711695,-79.416936,Menchie's St. Clair West,43.707664,-79.414301,Ice Cream Shop
1,Davisville North,0.0,0,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop
1,Davisville North,0.0,0,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
1,Davisville North,0.0,0,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
1,Davisville North,0.0,0,43.712751,-79.390197,Best Western Roehampton Hotel & Suites,43.708878,-79.390880,Hotel
1,Davisville North,0.0,0,43.712751,-79.390197,Circle K,43.712834,-79.391554,Grocery Store
1,Davisville North,0.0,0,43.712751,-79.390197,Gym,43.713126,-79.393537,Gym
1,Davisville North,0.0,0,43.712751,-79.390197,Winners,43.713236,-79.393873,Clothing Store


In [57]:
to_merged.loc[to_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,North Toronto West,0.125000,1,43.715383,-79.405678,Barreworks,43.714070,-79.400109,Yoga Studio
6,North Toronto West,0.125000,1,43.715383,-79.405678,Dufflet Pastries,43.715729,-79.400283,Dessert Shop
6,North Toronto West,0.125000,1,43.715383,-79.405678,Nailsense,43.717467,-79.400653,Spa
6,North Toronto West,0.125000,1,43.715383,-79.405678,C'est Bon,43.716785,-79.400406,Chinese Restaurant
6,North Toronto West,0.125000,1,43.715383,-79.405678,Starbucks,43.715456,-79.400303,Coffee Shop
6,North Toronto West,0.125000,1,43.715383,-79.405678,Tio's Urban Mexican,43.714630,-79.400000,Mexican Restaurant
6,North Toronto West,0.125000,1,43.715383,-79.405678,Uncle Betty's Diner,43.714452,-79.400091,Diner
6,North Toronto West,0.125000,1,43.715383,-79.405678,A&W,43.715124,-79.400017,Fast Food Restaurant
6,North Toronto West,0.125000,1,43.715383,-79.405678,Civello Salon,43.715111,-79.400304,Salon / Barbershop
2,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",0.142857,1,43.686412,-79.400049,Scallywags,43.687982,-79.394676,Pub


In [58]:
to_merged.loc[to_merged['Cluster Labels']==2]

,Neighborhood,Coffee Shop,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Davisville,0.052632,2,43.704324,-79.38879,Bread and Butter,43.701582,-79.387359,Sandwich Place
0,Davisville,0.052632,2,43.704324,-79.38879,Thai Spicy House,43.701962,-79.387513,Thai Restaurant
0,Davisville,0.052632,2,43.704324,-79.38879,Crossfit Metric,43.707480,-79.389857,Gym
0,Davisville,0.052632,2,43.704324,-79.38879,Hokkaido Sushi,43.708082,-79.389995,Sushi Restaurant
0,Davisville,0.052632,2,43.704324,-79.38879,Granite Brewery,43.707991,-79.389943,Brewery
0,Davisville,0.052632,2,43.704324,-79.38879,Florentia Ristorante,43.703594,-79.387985,Italian Restaurant
0,Davisville,0.052632,2,43.704324,-79.38879,Flaky Tart,43.706539,-79.389611,Dessert Shop
0,Davisville,0.052632,2,43.704324,-79.38879,Viva Napoli,43.705752,-79.389125,Pizza Place
0,Davisville,0.052632,2,43.704324,-79.38879,Positano,43.704558,-79.388639,Italian Restaurant
0,Davisville,0.052632,2,43.704324,-79.38879,June Rowlands Park,43.700517,-79.389189,Park


The cluster 0 is the cluster woth no coffee shops and the other cluster areas have coffee shops with some of them having large coffee shop chain stores like StarBucks. The best recommended area for the opening of a CCD store will be cluster 0 . Cluster 2 is the second best option with only 2 coffee shops. 